In [74]:
import os
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

In [75]:
## Load Dataset
dfDataSet = pd.read_csv("dataset.csv")

In [76]:
## Load Income Table and read it into a dictionary
dfIncome = pd.read_csv("dataset_income.csv")
dfIncome = dfIncome.reset_index(drop=True)
dict_Income = {}
for index, row in dfIncome.iterrows():
    dict_Income[int(row['Age'])] = int(row['Income'])

In [77]:
## Load Job Table and read it into a dictionary
dfJob = pd.read_csv("dataset_job.csv")
dfJob = dfJob.reset_index(drop=True)
dfJob.head()
dict_Job = {}
for index, row in dfJob.iterrows():
    dict_Job[row['Job']] = int(row['AverageSalary'])

In [78]:
## Extract only valid columns from dataset into new dataframe
validColumns = [col for col in dfDataSet if "Review" not in col]
dfDataSetNoReview = dfDataSet[validColumns]

In [79]:
## Remove dollar ($) symbol from price column
def RemoveDollarSymbol(col):
    value = col.values
    if '$' in str(value):
        return(col.map(lambda x: x.lstrip('$')))
    else:
        return(value)

## Get random age from between 18 to 75 years
def GetAge():
    return(random.randrange(18, 75))

## Get random gender
## 'gender' list/variable can also be changed to numeric such as gender = [1,2]
def GetGender():
    gender = ['Male','Female']
    random_index = random.randrange(len(gender))
    return(gender[random_index])

## Get Income from dict_Income based on 'Age' key
def GetIncome(dAge):
    randrange = random.randrange(5)
    dAge = int(dAge)
    if dAge in dict_Income:

        # if randomly selected number (randrange) is 0 or 5, multiply income by 2
        # ...just to make income range randomly diversified
        if randrange==0 or 5:
            return(dict_Income[dAge]*2)
        else:
            return(dict_Income[dAge])            
    else:
        return(10000)

## Get kind of Job based on Income
def GetJob(income):
    percDiff = income * 0.03
    minDiff = income - percDiff # minimum range
    maxDiff = income + percDiff # maximum range

    jobList = []
    for job,salary in dict_Job.items():
        # if salary falls within acceptable range, append job description to jobList 
        if salary >= minDiff and salary <= maxDiff: jobList.append(job)

    if len(jobList)==0:
        # if jobList is empty, then return "Other" as job title
        return('Other')
    elif len(jobList)==1:
        # if only 1 jobList, then return it as job title
        return(jobList[0])
    elif len(jobList)>1:
        # if jobList is more than 1, shuffle it and return 1st on the JobList
        random.shuffle(jobList)
        return(jobList[0])
        
## Randomly get zipcode between 80201 and 80239 (State of Colorado only)
def GetZipCode():
    randrange = random.randrange(80201,80239)
    return(randrange)


In [80]:
## If any of these Columns have null values, then drop the record
EmptyColumnCells_ToDrop = ['Audiobook_Type','Categories','Rating']
dfDataSetNoReview.dropna(subset=EmptyColumnCells_ToDrop, inplace=True)

## Remove dollar($) symbol from price column
dfDataSetNoReview['Price'] = RemoveDollarSymbol(dfDataSetNoReview['Price'])

## Convert Rating and Price columns to type 'Float'
dfDataSetNoReview[['Rating','Price']] = dfDataSetNoReview[['Rating','Price']].astype('float')
dfDataSetNoReview = dfDataSetNoReview.reset_index(drop=True)

In [81]:
## Initial columns to use with the 'Review' columns
initColumns = [col for col in dfDataSetNoReview]
## Add a single 'Review' column
initColumns.append("Review")

## All the new columns to add to the exploded dataset
extraColumns = ["Gender", "Age", "Income","ZipCode","Profession"]

In [82]:
## Initialize low and high rating list needed to populate the new exploded dataframe
low_rating = []
high_rating = []

## Start looping through dataframe
for index, row in dfDataSetNoReview.iterrows():

    # Get rating (<=5 max) from dataframe
    rating = row['Rating']

    # Only do something if it is of type integer/float 
    if type(rating) == int or float:
        
        # Percentage of good rating  ## Example rating divided by maximum rating (4.7/5) * 100
        rating_1 = int(((rating/5)*100))

        # Percentage of poor rating (remainder)
        rating_0 = 100 - rating_1
       
        dict_low_rating = {}
        dict_high_rating = {}
        
        # Loop thru low rating and append record to "dict_low_rating" dictionary
        # row['Review'] = 'Low' can also be changed to row['Review'] = 0
        for ndx in range(rating_0):
            row['Review'] = 'Low'
            for col in initColumns:
                dict_low_rating[col] = row[col]
            if 'Review' in dict_low_rating: low_rating.append(dict_low_rating)

        # Loop thru high rating and append record to "dict_high_rating" dictionary
        # row['Review'] = 'High' can also be changed to row['Review'] = 1                
        for ndx in range(rating_1):
            row['Review'] = 'High'
            for col in initColumns:
                dict_high_rating[col] = row[col]
            if 'Review' in dict_high_rating: high_rating.append(dict_high_rating)            

# concatenate the 2 list of records in 'high_rating' and 'low_rating'
# use it to populate a new dataframe (dfDataSetExploded) 
dfDataSetExploded = pd.DataFrame(high_rating + low_rating)

In [84]:
## If new dataframe (dfDataSetExploded) contain records, then
## ...fill in the blanks for Gender, Age, Income, ZipCode and Profession

if len(dfDataSetExploded) > 0:
    dfDataSetExploded[extraColumns] = None
    dfDataSetExploded = dfDataSetExploded.reset_index(drop=True)
    
    for index in range(len(dfDataSetExploded)):
        dfDataSetExploded.at[index,'Gender'] = GetGender()
        dfDataSetExploded.at[index,'Age'] = GetAge()
        dfDataSetExploded.at[index,'Income'] = GetIncome(dfDataSetExploded.at[index,'Age'])
        dfDataSetExploded.at[index,'ZipCode'] = GetZipCode()
        dfDataSetExploded.at[index,'Profession'] = GetJob(dfDataSetExploded.at[index,'Income'])

    ##  Write out new dataframe (dfDataSetExploded) to a csv
    dfDataSetExploded.to_csv('dataset_Exploded.csv', index=False)